#                     Projeto Final CDD- Prever a nota de um filme no IMDB

André Vital Tavernaro


Daniel Zelv Freund


Guilherme Batista

In [103]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import seaborn as sns
import numpy as np
from scipy.stats import norm, probplot
import statsmodels.api as sm
from mpl_toolkits.mplot3d import Axes3D
import json

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)

In [104]:
### Função utilitária para fazer a regressão com constante adicionada
def regress(X,Y):
    X_cp = sm.add_constant(X)
    model = sm.OLS(Y,X_cp)
    results = model.fit()
    return results

In [105]:
dados_filmes= pd.read_csv("tmdb_5000_movies.csv")

#### Tiramos os valures nulos do dataframe da coluna budget

In [122]:
dados_filmes.drop(dados_filmes[dados_filmes["budget"]==0].index,inplace=True)

## Construindo o Dataframe

In [107]:
dados_novos = dados_filmes[["title","runtime","budget","revenue", "vote_average", "vote_count",
                            "popularity","genres", 'release_date']].copy()

#### Criação de colunas com o nome dos generos, sendo 1 para quando o filme é daquela categoria e 0 para quando não é , isso foi feito para tirar a coluna genre

In [108]:
genres = set()
for x in dados_novos['genres']:
    for g in json.loads(x):
        genres.add(g["name"])

#### Criacão do novo dataframe incluindo as novas colunas de gênero

In [109]:
def extrai_generos(item):
    generos = {'c_' + x: 0 for x in ['Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 
                              'Documentary', 'Drama', 'Family', 'Fantasy', 'Foreign', 
                              'History', 'Horror', 'Music', 'Mystery', 'Romance', 
                              'Science Fiction', 'TV Movie', 'Thriller', 'War', 'Western']}
    for g in json.loads(item):
        generos['c_' + g['name']] += 1
    return generos

dados_novos_teste = pd.DataFrame(list(dados_novos['genres'].apply(extrai_generos).values))

dados_novos_junto = dados_novos.join(dados_novos_teste)

dados_novos_junto.set_index("title",inplace=True)

In [110]:
dados_novos_junto.drop(columns= "genres", inplace=True)

#### Criação de novas colunas que contém o mês, ano, e dia , cada uma em sua própia coluna no dataframe 

In [111]:
dados_novos_junto['ano'] , dados_novos_junto['mes'] , dados_novos_junto['dia'] = dados_novos_junto['release_date'].str.split('-',2).str
dados_novos_junto['ano'] = pd.to_numeric(dados_novos_junto['ano'])
dados_novos_junto['mes'] = pd.to_numeric(dados_novos_junto['mes'])
dados_novos_junto['dia']  = pd.to_numeric(dados_novos_junto['dia'] )

## Variáveis para a análise

**Dicionário de dados:**


Variável | Descrição
:---:|---:
title | Título do filme
runtime | Tempo de filme
budget | Orçamento
revenue | Receita
vote_average | Média dos votos
vote_count | Número de votos das pessoas
popularity | Popularidade
c_Action | 0 quando o filme não pertence à esse gênero, e 1 quando pertence
c_Adventure | 0 quando o filme não pertence à esse gênero, e 1 quando pertence
c_Animation | 0 quando o filme não pertence à esse gênero, e 1 quando pertence
c_Comedy | 0 quando o filme não pertence à esse gênero, e 1 quando pertence
c_Crime | 0 quando o filme não pertence à esse gênero, e 1 quando pertence
c_Documentary | 0 quando o filme não pertence à esse gênero, e 1 quando pertence
c_Drama | 0 quando o filme não pertence à esse gênero, e 1 quando pertence
c_Family | 0 quando o filme não pertence à esse gênero, e 1 quando pertence
c_Fantasy | 0 quando o filme não pertence à esse gênero, e 1 quando pertence
c_Foreign | 0 quando o filme não pertence à esse gênero, e 1 quando pertence
c_History | 0 quando o filme não pertence à esse gênero, e 1 quando pertence
c_Horror | 0 quando o filme não pertence à esse gênero, e 1 quando pertence
c_Music | 0 quando o filme não pertence à esse gênero, e 1 quando pertence
c_Mystery | 0 quando o filme não pertence à esse gênero, e 1 quando pertence
c_Romance | 0 quando o filme não pertence à esse gênero, e 1 quando pertence
c_Science Fiction | 0 quando o filme não pertence à esse gênero, e 1 quando pertence
c_TV Movie | 0 quando o filme não pertence à esse gênero, e 1 quando pertence
c_Thriller | 0 quando o filme não pertence à esse gênero, e 1 quando pertence
c_War | 0 quando o filme não pertence à esse gênero, e 1 quando pertence
c_Western | 0 quando o filme não pertence à esse gênero, e 1 quando pertence
ano | ano em que o filme foi lançado
mês | mês em que o filme foi lançado
dia | dia em que o filme foi lançado

### Verificando e limpando os  valores nulos e inválidos das variáveis em certa coluna

In [123]:
np.sum(df.isnull())

runtime              0
budget               0
revenue              0
vote_average         0
vote_count           0
popularity           0
release_date         0
c_Action             0
c_Adventure          0
c_Animation          0
c_Comedy             0
c_Crime              0
c_Documentary        0
c_Drama              0
c_Family             0
c_Fantasy            0
c_Foreign            0
c_History            0
c_Horror             0
c_Music              0
c_Mystery            0
c_Romance            0
c_Science Fiction    0
c_TV Movie           0
c_Thriller           0
c_War                0
c_Western            0
ano                  0
mes                  0
dia                  0
dtype: int64

In [124]:
df = dados_novos_junto

df = df.dropna()

## Com a ajuda do comando Describe, poderemos analisar fatores importantes do Dataframe

In [112]:
dados_novos_junto.describe()

,runtime,budget,revenue,vote_average,vote_count,popularity,c_Action,c_Adventure,c_Animation,c_Comedy,...,c_Mystery,c_Romance,c_Science Fiction,c_TV Movie,c_Thriller,c_War,c_Western,ano,mes,dia
count,3764.000000,3.766000e+03,3.766000e+03,3766.000000,3766.000000,3766.000000,3221.000000,3221.000000,3221.000000,3221.000000,...,3221.000000,3221.000000,3221.000000,3221.000000,3221.000000,3221.000000,3221.000000,3766.000000,3766.000000,3766.000000
mean,109.324389,3.704284e+07,1.039547e+08,6.226474,856.496814,25.984641,0.282521,0.208010,0.060540,0.350512,...,0.074201,0.174170,0.127600,0.000931,0.289662,0.035082,0.017075,2002.090813,6.935741,15.433617
std,20.753408,4.264651e+07,1.776399e+08,0.966228,1344.785690,34.438253,0.450295,0.405947,0.238522,0.477204,...,0.262138,0.379314,0.333696,0.030509,0.453676,0.184016,0.129573,12.876561,3.393160,8.520344
min,0.000000,1.000000e+00,0.000000e+00,0.000000,0.000000,0.001586,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1916.000000,1.000000,1.000000
25%,95.000000,8.000000e+06,6.009736e+06,5.700000,114.000000,7.974990,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1999.000000,4.000000,9.000000
50%,105.500000,2.300000e+07,3.883137e+07,6.300000,365.500000,17.604955,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2005.000000,7.000000,15.000000
75%,120.000000,5.000000e+07,1.220873e+08,6.900000,969.250000,33.925801,1.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,2011.000000,10.000000,22.000000
max,338.000000,3.800000e+08,2.787965e+09,10.000000,13752.000000,875.581305,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2016.000000,12.000000,31.000000


## Uso da função Regress

In [117]:
results = regress(X,Y)
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           vote_average   R-squared:                       0.378
Model:                            OLS   Adj. R-squared:                  0.374
Method:                 Least Squares   F-statistic:                     88.28
Date:                Mon, 12 Nov 2018   Prob (F-statistic):          5.52e-309
Time:                        16:20:53   Log-Likelihood:                -3351.0
No. Observations:                3220   AIC:                             6748.
Df Residuals:                    3197   BIC:                             6888.
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                20.6481      2.596      7.955      0.000      15.559      25.737
runtime               0.0144      0.001     22.121      0.000       0.013       0.016
budget            -6.174e-09   4.31e-10    -14.337      0.000   -7.02e-09   -5.33e-09
revenue           -2.735e-11   1.19e-10     -0.229      0.819   -2.61e-10    2.07e-10
vote_count            0.0003   1.62e-05     17.864      0.000       0.000       0.000
popularity            0.0012      0.001      2.383      0.017       0.000       0.002
c_Action             -0.0122      0.033     -0.373      0.709      -0.076       0.052
c_Adventure          -0.0947      0.035     -2.700      0.007      -0.164      -0.026
c_Animation           0.2341      0.055      4.263      0.000       0.126       0.342
c_Comedy             -0.0103      0.029     -0.358      0.720      -0.067       0.046
c_History            -0.1066      0.064     -1.662      0.097      -0.232       0.019
c_Horror              0.0419      0.042      0.993      0.321      -0.041       0.125
c_Music              -0.0685      0.067     -1.017      0.309      -0.201       0.064
c_Mystery            -0.0489      0.049     -0.999      0.318      -0.145       0.047
c_Romance            -0.0130      0.034     -0.380      0.704      -0.080       0.054
c_Science Fiction     0.0111      0.039      0.286      0.775      -0.065       0.087
c_TV Movie            0.3847      0.398      0.966      0.334      -0.396       1.165
c_Thriller           -0.0267      0.032     -0.833      0.405      -0.090       0.036
c_War                -0.0706      0.071     -0.993      0.321      -0.210       0.069
c_Western             0.0210      0.095      0.220      0.826      -0.166       0.208
mes                   0.0212      0.004      5.868      0.000       0.014       0.028
ano                  -0.0081      0.001     -6.262      0.000      -0.011      -0.006
dia                  -0.0011      0.001     -0.732      0.464      -0.004       0.002
==============================================================================
Omnibus:                      275.665   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              461.872
Skew:                          -0.624   Prob(JB):                    5.08e-101
Kurtosis:                       4.374   Cond. No.                     4.87e+10
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.87e+10. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Criação do novo dataframe com variáveis releventes

#### Adotamos um ALFA de 5%, então só vamos usar na análise as variáveis  que possuem um P>T maior que o ALFA 

Obtendo as variáveis resposta e preditora:

-Variável resposta (Y) , ou seja , as notas do site IMDB

-Variável preditora (X) , ou seja , todas as colunas que serão usadas para prever as notas dos filmes

In [115]:
Y = df[ "vote_average"]

In [118]:
X = df[["runtime","budget","vote_count", "popularity",'mes','ano']]

#### Separando o dataframe em teste e treinamento

In [125]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=RANDOM_SEED)

 #### Aplicando o modelo de regressão linear

In [126]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

model = LinearRegression()       #cria
model.fit(X_train, y_train)        #treina 
y_pred = model.predict(X_test)      #testa 

resultrl = 1 - mean_squared_error(y_test, y_pred) / np.var(y_test)
print('a margem de acerto do modelo de regressão linear é de {0}%'.format(resultrl))

a margem de acerto do modelo de regressão linear é de 0.3879962273565626%


#### Aplicando o modelo RandomForest

In [127]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

model = RandomForestRegressor()       #cria
model.fit(X_train, y_train)        #treina 
y_pred = model.predict(X_test)      #testa 

resultrf = 1 - mean_squared_error(y_test, y_pred) / np.var(y_test)
print('a margem de acerto do modelo RandomForest é de {0}%'.format(resultrf))

a margem de acerto do modelo RandomForest é de 0.45872795029420144%


C:\Users\andre\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
